In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

# Background
## Simulating stock market movement over quarters with varied market character; can our agents successfully predict market price for all of them?

In [9]:
# Let's create dataframes for each quarter
# 1. Get all news
all_news_df = pd.read_csv("dataset/analyst_ratings_processed.csv")

print(f"Total articles before date cleaning: {len(all_news_df)}")

# 2. Convert to datetime, forcing errors to NaT (Not a Time)
all_news_df['date'] = pd.to_datetime(all_news_df['date'], errors='coerce', utc=True)

# 3. Drop the rows where the date was invalid
all_news_df = all_news_df.dropna(subset=['date'])

print(f"Total articles after date cleaning: {len(all_news_df)}")

# 4. Create a 'quarter' helper column (format: '2009Q2')
all_news_df['q_index'] = all_news_df['date'].dt.to_period('Q').astype(str)

Total articles before date cleaning: 1400469
Total articles after date cleaning: 1397891


C:\Users\shara\AppData\Local\Temp\ipykernel_45560\2345186197.py:16: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  all_news_df['q_index'] = all_news_df['date'].dt.to_period('Q').astype(str)


In [10]:
# Let's look at how much data we have for each quarter
articles_by_q = all_news_df['q_index'].value_counts(dropna=False).reset_index()
print(articles_by_q)

   q_index  count
0   2020Q1  59849
1   2020Q2  45826
2   2019Q4  42475
3   2019Q3  41954
4   2018Q3  41612
5   2018Q4  41611
6   2014Q4  36368
7   2016Q1  35906
8   2016Q3  35536
9   2011Q3  35318
10  2015Q1  35066
11  2015Q2  34961
12  2016Q4  34943
13  2016Q2  34929
14  2011Q2  34284
15  2010Q4  34223
16  2019Q2  33800
17  2014Q3  32911
18  2012Q1  32570
19  2015Q4  32098
20  2013Q1  32064
21  2017Q1  32047
22  2018Q2  32004
23  2019Q1  31851
24  2014Q2  31590
25  2011Q4  31438
26  2017Q4  31314
27  2011Q1  31302
28  2018Q1  31186
29  2013Q2  31117
30  2012Q2  30926
31  2015Q3  30752
32  2013Q3  29751
33  2017Q2  29664
34  2012Q4  29649
35  2014Q1  29082
36  2012Q3  29067
37  2013Q4  28340
38  2017Q3  27274
39  2010Q3  20648
40  2010Q2  13675
41  2010Q1  12590
42  2009Q3   7590
43  2009Q4   6677
44  2009Q2     52
45  2009Q1      1


# Based on the above results, have decided on the following quarters:
* 2012Q1
> 32,570 articles, can we beat a basic bull market?
* 2014Q4
> 36,368 articles, can we handle sector-specific shocks (Oil)?
* 2016Q1
> 35,906 articles, can we survive a 10% dip and stay in the game?
* 2018Q4
> 41,611 articles, can we manage a sharp, high-volume year-end crash?

In [11]:
# What stocks do we need to keep track of for each quarter? Let's split up the original df to find out.
q1_2012 = all_news_df[all_news_df['q_index'] == '2012Q1'].copy()
q4_2014 = all_news_df[all_news_df['q_index'] == '2014Q4'].copy()
q1_2016 = all_news_df[all_news_df['q_index'] == '2016Q1'].copy()
q4_2018 = all_news_df[all_news_df['q_index'] == '2018Q4'].copy()

# Verify counts
print(len(q1_2012), len(q4_2014), len(q1_2016), len(q4_2018))

32570 36368 35906 41611


In [12]:
# Count stock tickers for each quarter

q1_2012_stocks = q1_2012['stock'].value_counts(dropna=False).reset_index()
q4_2014_stocks = q4_2014['stock'].value_counts(dropna=False).reset_index()
q1_2016_stocks = q1_2016['stock'].value_counts(dropna=False).reset_index()
q4_2018_stocks = q4_2018['stock'].value_counts(dropna=False).reset_index()

print(q1_2012_stocks)
print(q4_2014_stocks)
print(q1_2016_stocks)
print(q4_2018_stocks)

     stock  count
0      QQQ    538
1      NBG    329
2     ZNGA    151
3     VVUS    145
4     GMCR    139
...    ...    ...
3352  SNAK      1
3353  SMTX      1
3354   SMN      1
3355   CTV      1
3356   CTT      1

[3357 rows x 2 columns]
     stock  count
0     GPRO    224
1     TKMR    220
2     LAKE    188
3     EBAY    177
4     NLNK    177
...    ...    ...
4116  ABCW      1
4117  EMLP      1
4118  XNET      1
4119  XNCR      1
4120   XME      1

[4121 rows x 2 columns]
     stock  count
0     GPRO    253
1      CMG    220
2     YHOO    211
3      TWX    146
4     BABA    135
...    ...    ...
3995   KHI      1
3996   KEM      1
3997  EUDG      1
3998   ETM      1
3999  PFNX      1

[4000 rows x 2 columns]
      stock  count
0       DIA    329
1      NFLX    178
2     GOOGL    176
3       XRT    174
4       EWW    163
...     ...    ...
3732   SLCT      1
3733    SRT      1
3734    DBV      1
3735    DNN      1
3736    DKT      1

[3737 rows x 2 columns]


In [13]:
# Cost Analysis: How many tokens will be used to embed all of these news articles?

# Function to count words safely
def count_words(text):
    if isinstance(text, str):
        # Split by whitespace and filter out empty strings
        return len([word for word in text.strip().split() if word])
    return 0  # For NaN or non-string values

# Apply function to column
q4_2018['word_count'] = q4_2018['title'].apply(count_words) + 1 
q1_2016['word_count'] = q1_2016['title'].apply(count_words) + 1 
q4_2014['word_count'] = q4_2014['title'].apply(count_words) + 1 
q1_2012['word_count'] = q1_2012['title'].apply(count_words) + 1 

# Total number of words in the column
total_words = q4_2018['word_count'].sum() + q1_2016['word_count'].sum() + q4_2014['word_count'].sum() + q1_2012['word_count'].sum()

avg_tk_per_wd = 4
cost_per_m_tokens = 0.15
projected_cost = total_words * avg_tk_per_wd * (cost_per_m_tokens / 1000000)
total_words, projected_cost

(np.int64(1796405), np.float64(1.0778429999999999))

In [ ]:
# How long is the longest document? If it's just headlines, can we embed without chunking?
max_q4_2018 = q4_2018['word_count'].max()
max_q1_2016 = q1_2016['word_count'].max()
max_q4_2014 = q4_2014['word_count'].max()
max_q1_2012 = q1_2012['word_count'].max()
print(max(max_q4_2018, max_q1_2016, max_q4_2014, max_q1_2012)) # 59 words

59
